In [3]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split as tts
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,matthews_corrcoef as mat,confusion_matrix,roc_auc_score,classification_report
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler

from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt

In [4]:
df=pd.read_csv("parkinsons.csv")

In [5]:
x=df.drop(['status','name'],axis=1)
norm = MinMaxScaler().fit(x)
X= norm.transform(x)

y=df['status']
x_train,x_test,y_train,y_test=tts(X,y,test_size=0.3,random_state=10)

In [6]:
bestfeatures = SelectKBest(score_func=chi2, k=12)
fit = bestfeatures.fit(x_train,y_train)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(x.columns)
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score'] 
featureScores['Specs'].values
print(featureScores.nlargest(12,'Score')) 

               Specs     Score
2       MDVP:Flo(Hz)  3.355342
18           spread1  3.308019
21               PPE  3.150308
10      Shimmer:APQ3  2.656304
13       Shimmer:DDA  2.655040
8       MDVP:Shimmer  2.622205
11      Shimmer:APQ5  2.417668
9   MDVP:Shimmer(dB)  2.339823
12          MDVP:APQ  1.996787
19           spread2  1.455480
16              RPDE  1.425923
6           MDVP:PPQ  1.389665


In [7]:
from imblearn.combine import SMOTETomek
smk = SMOTE(random_state=5)
X_res,y_res=smk.fit_sample(x_train,y_train)
sns.countplot(x=y_res)

AttributeError: 'SMOTE' object has no attribute 'fit_sample'

In [8]:
new_model=CatBoostClassifier(depth=10, iterations=70, learning_rate= 0.04)
new_model.fit(X_res, y_res)

NameError: name 'X_res' is not defined

In [66]:
y_pred=new_model.predict(x_test)
acc2=accuracy_score(y_test,y_pred)
acc2

0.9322033898305084

In [67]:
print(classification_report(y_test,y_pred))
math2=mat(y_test,y_pred)
math2

              precision    recall  f1-score   support

           0       0.88      0.88      0.88        16
           1       0.95      0.95      0.95        43

    accuracy                           0.93        59
   macro avg       0.91      0.91      0.91        59
weighted avg       0.93      0.93      0.93        59



0.8284883720930233

In [68]:
xgb=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0.0,
              learning_rate=0.2, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)
xgb.fit(X_res,y_res)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0.0,
              learning_rate=0.2, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [69]:
predict_xgb=xgb.predict(x_test)
accu_xgb=accuracy_score(y_test,predict_xgb)

In [70]:
accu_xgb

0.9491525423728814

In [71]:
print(classification_report(y_test,predict_xgb))
math2=mat(y_test,predict_xgb)
math2

              precision    recall  f1-score   support

           0       0.88      0.94      0.91        16
           1       0.98      0.95      0.96        43

    accuracy                           0.95        59
   macro avg       0.93      0.95      0.94        59
weighted avg       0.95      0.95      0.95        59



0.8746154593939881

In [78]:
model=RandomForestClassifier(criterion='gini',max_depth= 9, n_estimators= 200)
model.fit(X_res,y_res)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=9, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [99]:

modelExtra=ExtraTreesClassifier()
modelExtra.fit(x_train,y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
                     max_depth=None, max_features='auto', max_leaf_nodes=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
                     oob_score=False, random_state=None, verbose=0,
                     warm_start=False)

In [100]:

y_predExtra=modelExtra.predict(x_test)
print(accuracy_score(y_test, y_predExtra)*100)

94.91525423728814
